# ARHGAP5 Mutation Trans Effect on Transcriptomics

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings("ignore")

import cptac
import cptac.utils as al

brain = cptac.Gbm()

### Select Gene

In [2]:
gene = "ARHGAP5"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
omics = "transcriptomics"
#omics = "phosphoproteomics"
#omics = "acetylproteomics"

### Track all significant comparisons in Dataframe

In [4]:
all_significant_comparisons = pd.DataFrame(columns=['Cancer_Type', 'Gene', 'Comparison','Interacting_Protein','P_Value'])

In [5]:
def add_to_all_significant_comparisons(df, cancer, interacting, all_sig_comp):
    expanded = df
    expanded['Gene'] = gene
    expanded['Cancer_Type'] = cancer
    expanded['Interacting_Protein'] = interacting
    
    updated_all_comparisons = pd.concat([all_sig_comp, expanded], sort=False)
    
    return updated_all_comparisons

# Interacting Proteins: Transcriptomics

### Generate interacting protein list

Make a call to cptac.utils to get the interacting proteins method, which interacts with the uniprot and string databases to generate a list of known interacting partners with the given gene.

In [6]:
# Use get interacting proteins method to generate list of interacting proteins
interacting_proteins = al.get_interacting_proteins(gene)

# Show interacting protein list
print("Interacting Proteins:")
for interacting_protein in interacting_proteins:
    print(interacting_protein)

Interacting Proteins:
RHOV
ARHGEF5
RHOQ
RAC2
RHOF
RHOB
RHOC
RAC3
RHOD
RHOJ
RHOT2
RHOBTB1
RHOG
RAC1
RHOT1
RHOU
SRC
RHOH
ARHGAP5
ROCK1
CDC42
RHOA
HRAS
RHOBTB2
ARF1
VAV1
RND3
RND1
RND2
XRCC6
SMURF1
SMURF2
NPM1
ADRB2
UNK
CDH1
PTPRU
IQCF1
PLK1
RNF123


### Test for significant comparisons in any of interacting proteins

In [7]:
# Create dataframe in order to do comparisons with wrap_ttest
protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins)
protdf = brain.reduce_multiindex(protdf, flatten=True)
protdf = protdf[protdf.Sample_Status == 'Tumor'] # drop Normal samples

# Create the binary valued column needed to do the comparison
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

# Format the dataframe correctly
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


# Make list of columns to be compared using t-tests
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

# Call wrap_ttest, pass in formatted dataframe
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

# Print results, if anything significant was found
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", True, all_significant_comparisons)
        

Doing t-test comparisons

No significant comparisons.


# All Proteins: Transcriptomics 

In [8]:
try:
    print("\nGene: ", gene)

    # Use all proteins

    # Create dataframe in order to do comparisons with wrap_ttest
    protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics)
    protdf = brain.reduce_multiindex(protdf, flatten=True) # flatten for specific column names
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    # Create binary column 
    protdf['Label'] = np.where(
                protdf[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

    # Format the dataframe correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    # Make list of columns to be compared using t-tests
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    # Call wrap_ttest, pass in formatted dataframe
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    # Print results, if anything significant was found
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")
            
            all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", False, all_significant_comparisons)


except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  ARHGAP5


Doing t-test comparisons

                                         Comparison       P_Value
0           ATP4B_transcriptomics_ENSG00000186009.4  6.153592e-10
1   RP11-386G11.3_transcriptomics_ENSG00000258283.1  1.288919e-08
2            DRGX_transcriptomics_ENSG00000165606.7  1.438286e-08
3      AC012354.6_transcriptomics_ENSG00000225156.2  2.146154e-08
4         LRRC10B_transcriptomics_ENSG00000204950.3  2.262499e-08
5         MYB-AS1_transcriptomics_ENSG00000236703.1  2.826756e-08
6          uc_338_transcriptomics_ENSG00000275001.1  2.838073e-08
7    RP11-316I3.2_transcriptomics_ENSG00000238054.1  2.923351e-08
8    RP11-351C8.1_transcriptomics_ENSG00000253573.1  3.252915e-08
9   CTD-2311M21.2_transcriptomics_ENSG00000260266.1  3.395080e-08
10  RP11-474L11.3_transcriptomics_ENSG00000220730.2  3.825574e-08
11   RP11-313I2.6_transcriptomics_ENSG00000255170.2  3.964755e-08
12      RNU6-199P_transcriptomics_ENSG00000199824.1  4.160904e-08
13         TBX20_transcriptomics_ENSG00000164532.1

# Print all significant comparisons

In [9]:
if len(all_significant_comparisons) > 0:
    display(all_significant_comparisons)
    
else:
    print('No Significant Comparisons!')

,Cancer_Type,Gene,Comparison,Interacting_Protein,P_Value
0,Gbm,ARHGAP5,ATP4B_transcriptomics_ENSG00000186009.4,False,6.153592e-10
1,Gbm,ARHGAP5,RP11-386G11.3_transcriptomics_ENSG00000258283.1,False,1.288919e-08
2,Gbm,ARHGAP5,DRGX_transcriptomics_ENSG00000165606.7,False,1.438286e-08
3,Gbm,ARHGAP5,AC012354.6_transcriptomics_ENSG00000225156.2,False,2.146154e-08
4,Gbm,ARHGAP5,LRRC10B_transcriptomics_ENSG00000204950.3,False,2.262499e-08
5,Gbm,ARHGAP5,MYB-AS1_transcriptomics_ENSG00000236703.1,False,2.826756e-08
6,Gbm,ARHGAP5,uc_338_transcriptomics_ENSG00000275001.1,False,2.838073e-08
7,Gbm,ARHGAP5,RP11-316I3.2_transcriptomics_ENSG00000238054.1,False,2.923351e-08
8,Gbm,ARHGAP5,RP11-351C8.1_transcriptomics_ENSG00000253573.1,False,3.252915e-08
9,Gbm,ARHGAP5,CTD-2311M21.2_transcriptomics_ENSG00000260266.1,False,3.395080e-08


### Write Significant Comparisons (if any) to Shared CSV file

In [10]:
'''
existing_results = pd.read_csv(gene+'_Trans_Results.csv')

updated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)

updated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)'''

"\nexisting_results = pd.read_csv(gene+'_Trans_Results.csv')\n\nupdated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)\n\nupdated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)"